In [38]:
from langchain_openai import ChatOpenAI

from extract import (
    extract_function_names,
    extract_function_by_name,
    extract_imports,
)

from llm_calls import (
    llm_generate_unit_tests,
    llm_generate_function_from_unit_tests,
    llm_find_bug,
    llm_fix_bug,
)

from run_tests import (
    find_first_failing_unit_test,
    run_unit_test,
)

from function_code_with_unit_tests import FunctionCodeWithUnitTests

llm = ChatOpenAI(temperature=0, model="gpt-4o")

# query = "Extract emails (including instances of 'name at email dot com') from a text."
# query = "Swap the words 'cat' and 'dog' in the text."

query = "For a given string, please extract all the email addresses. \
Including instances of 'name at email dot com', for example 'cat at gmail dot com' should extract 'cat@gmail.com' \
and 'dog at hotmail dot com' should extract 'dog@hotmail.com'. Should should work for any email domain."

In [39]:
unit_test_code_as_string = llm_generate_unit_tests(llm, query)
print(unit_test_code_as_string)

# Function to test: extract_emails

import re
import pytest

def test_extract_emails_basic():
    text = "Please contact us at support@example.com for further information."
    expected = ["support@example.com"]
    result = extract_emails(text)
    assert result == expected, f"Expected {expected}, but got {result}"

def test_extract_emails_multiple():
    text = "Send an email to admin@example.com or sales@example.com."
    expected = ["admin@example.com", "sales@example.com"]
    result = extract_emails(text)
    assert result == expected, f"Expected {expected}, but got {result}"

def test_extract_emails_with_at_dot_format():
    text = "You can reach me at john.doe at gmail dot com or jane.doe at yahoo dot com."
    expected = ["john.doe@gmail.com", "jane.doe@yahoo.com"]
    result = extract_emails(text)
    assert result == expected, f"Expected {expected}, but got {result}"

def test_extract_emails_mixed_formats():
    text = "Contact us at support@example.com or john.doe at gmail 

In [40]:
unit_test_function_names = extract_function_names(code_string=unit_test_code_as_string)
unit_test_function_names

['test_extract_emails_basic',
 'test_extract_emails_multiple',
 'test_extract_emails_with_at_dot_format',
 'test_extract_emails_mixed_formats',
 'test_extract_emails_no_emails',
 'test_extract_emails_invalid_emails',
 'test_extract_emails_special_characters',
 'test_extract_emails_unicode_characters',
 'test_extract_emails_case_insensitive',
 'test_extract_emails_multiline',
 'test_extract_emails_non_matching_scenarios',
 'test_extract_emails_edge_case_empty_string',
 'test_extract_emails_edge_case_only_spaces',
 'test_extract_emails_edge_case_special_format',
 'test_extract_emails_large_input']

In [41]:
unit_test_imports = extract_imports(unit_test_code_as_string)
unit_test_imports

['import re', 'import pytest']

In [42]:
target_function_code_as_string = llm_generate_function_from_unit_tests(
    llm=llm,
    unit_test_code_as_string=unit_test_code_as_string,
    query=query,
)
print(target_function_code_as_string)

import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts all email addresses from the given text. This includes both standard email formats
    and those written in the 'name at domain dot com' format.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    standard_email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    at_dot_email_pattern = r'([a-zA-Z0-9._%+-]+) at ([a-zA-Z0-9.-]+) dot ([a-zA-Z]{2,})'
    
    standard_emails = re.findall(standard_email_pattern, text)
    at_dot_emails = re.findall(at_dot_email_pattern, text)
    
    converted_emails = [f"{user}@{domain}.{tld}" for user, domain, tld in at_dot_emails]
    
    return standard_emails + converted_emails



In [43]:
function_code_with_unit_tests = FunctionCodeWithUnitTests(
    function_code=[target_function_code_as_string],
    unit_tests={
        name: [
            extract_function_by_name(
                code_string=unit_test_code_as_string, function_name=name
            )
        ]
        for name in unit_test_function_names
    },
    unit_test_imports="\n".join(unit_test_imports)
)


In [44]:
for i in range(10):

    first_failing_unit_test_message = find_first_failing_unit_test(
        function_code_with_unit_tests=function_code_with_unit_tests
    )

    first_failing_unit_test_message.print_message_details()

    if first_failing_unit_test_message.test_pass_status:
        print("All tests are passing")
        break

    print("Ask llm if the function or unit test needs to be updated.")
    bug_report_dict = llm_find_bug(
        llm=llm,
        query=query,
        function_code_with_unit_tests=function_code_with_unit_tests,
        first_failing_unit_test_message=first_failing_unit_test_message,
    )

    if bug_report_dict["Likely Bug Location"] == "Unit Test":
        
        print("The llm said the unit test should be updated.\nAsk the llm to fix the unit test.\n")
        updated_unit_test_code = llm_fix_bug(
            llm=llm,
            query=query,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        
        print("Unit test before update;")
        print(function_code_with_unit_tests.get_unit_test_code(first_failing_unit_test_message.unit_test_name) + "\n")
        
        function_code_with_unit_tests.replace_unit_test(
            unit_test_name=first_failing_unit_test_message.unit_test_name,
            new_unit_test_code=updated_unit_test_code,
        )
        
        print("Unit test after update;")
        print(function_code_with_unit_tests.get_unit_test_code(first_failing_unit_test_message.unit_test_name) + "\n")

    elif bug_report_dict["Likely Bug Location"] == "Function":
        
        print("The llm said the function should be updated.\nAsk the llm to fix the function.\n")
        updated_function_code = llm_fix_bug(
            llm=llm,
            query=query,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        
        print("Function before update;")
        print(function_code_with_unit_tests.get_function_code() + "\n")
        
        function_code_with_unit_tests.replace_function_code(
            new_function_code=updated_function_code,
        )
        
        print("Function after update;")
        print(function_code_with_unit_tests.get_function_code() + "\n")
        
    else:
        raise ValueError("bug_report_dict keys not as expected")

    unit_test_message = run_unit_test(
        function_code_with_unit_tests=function_code_with_unit_tests,
        unit_test_name=first_failing_unit_test_message.unit_test_name,
    )
    
    if unit_test_message.test_pass_status:
    
        print(f"Unit test {first_failing_unit_test_message.unit_test_name}() now passing.")
    
    else:
    
        unit_test_code_to_delete = function_code_with_unit_tests.get_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        print(
            f"Unable to make this unit test pass\n{unit_test_code_to_delete}It will be deleted\n"
        )
        function_code_with_unit_tests.delete_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        if bug_report_dict["Likely Bug Location"] == "Function":
            print("Roll back update to function.")
            function_code_with_unit_tests.delete_function_changes()


if first_failing_unit_test_message.test_pass_status == False:
    print("Some tests are still not passing")

unit test test_extract_emails_basic() is currently passing. 1/15 tests pass.
unit test test_extract_emails_multiple() is currently passing. 2/15 tests pass.
unit test test_extract_emails_with_at_dot_format() is currently passing. 3/15 tests pass.
unit test test_extract_emails_mixed_formats() is currently passing. 4/15 tests pass.
unit test test_extract_emails_no_emails() is currently passing. 5/15 tests pass.
unit test test_extract_emails_invalid_emails() is currently passing. 6/15 tests pass.
unit test test_extract_emails_special_characters() is currently passing. 7/15 tests pass.

test_pass_status: False
unit_test_name: test_extract_emails_unicode_characters
output: Assertion failed: Expected ['üser@example.com'], but got ['ser@example.com']

Ask llm if the function or unit test needs to be updated.
The llm said the function should be updated.
Ask the llm to fix the function.

Function before update;
import re
from typing import List

def extract_emails(text: str) -> List[str]:
    "

In [45]:
print(function_code_with_unit_tests.get_function_code())

import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts all email addresses from the given text. This includes both standard email formats
    and those written in the 'name at domain dot com' format.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    standard_email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    at_dot_email_pattern = r'([a-zA-Z0-9._%+-]+) at ((?:[a-zA-Z0-9-]+ dot )+[a-zA-Z]{2,})'
    
    standard_emails = re.findall(standard_email_pattern, text)
    at_dot_emails = re.findall(at_dot_email_pattern, text)
    
    converted_emails = [f"{user}@{domain.replace(' dot ', '.')}" for user, domain in at_dot_emails]
    
    return standard_emails + converted_emails



In [46]:
for unit_test_name in function_code_with_unit_tests.unit_tests:
    print(
        function_code_with_unit_tests.get_unit_test_code(unit_test_name=unit_test_name)
    )

def test_extract_emails_basic():
    text = 'Please contact us at support@example.com for further information.'
    expected = ['support@example.com']
    result = extract_emails(text)
    assert result == expected, f'Expected {expected}, but got {result}'

def test_extract_emails_multiple():
    text = 'Send an email to admin@example.com or sales@example.com.'
    expected = ['admin@example.com', 'sales@example.com']
    result = extract_emails(text)
    assert result == expected, f'Expected {expected}, but got {result}'

def test_extract_emails_with_at_dot_format():
    text = (
        'You can reach me at john.doe at gmail dot com or jane.doe at yahoo dot com.'
        )
    expected = ['john.doe@gmail.com', 'jane.doe@yahoo.com']
    result = extract_emails(text)
    assert result == expected, f'Expected {expected}, but got {result}'

def test_extract_emails_mixed_formats():
    text = 'Contact us at support@example.com or john.doe at gmail dot com.'
    expected = ['support@exampl